Definiamo la struttura del post a partire dalla ricetta, che si presenterà in questo modo
```
{
    _id: #,
    _idUser: #, 
    username: String, [R]
    description: String,
    timestamp: Timestamp,
    recipe: {
                name: String,
                image: String_URL,
                steps: [String, ...],
                totalCalories: Double, [R]
                ingredients: [{
                                name: String,
                                quantity: Double, 
                }, ...]
    },
    avgStarRanking: Double, [R]
    starRankings: [{
                        _idUser: #,
                        username: String, [R]
                        vote: Double,
                        (timestamp: Timestamp)
    }, ...],
    comments: [{
                _idUser: #,
                username: String, [R]
                text: String,
                timestamp: Timestamp
    }, ...]
}
```

In [2]:
import json

In [ ]:
def calcola_calorie_totali(ingredienti):
    total_calories = 0
    for ingrediente in ingredienti:
        total_calories += (ingrediente["quantity"] / 100) * ingrediente["Cals_per100grams"]
    return round(total_calories, 2)

def trasforma_ricette_in_post(file_ricette, file_output):
    # Leggi il file delle ricette
    with open(file_ricette, 'r', encoding='utf-8') as file:
        ricette_json = json.load(file)

    post_array = []

    for ricetta in ricette_json:
        total_calories = calcola_calorie_totali(ricetta["ingredients"])
        post = {
            "id": ricetta["id"],
            "idUser": ricetta["contributor_id"],
            "description": ricetta["description"],
            "timestamp": ricetta["submitted"],
            "recipe": {
                "name": ricetta["name"],
                "image": ricetta["Image"],
                "steps": ricetta["steps"],
                "ingredients": ricetta["ingredients"],
                "totalCalories": total_calories
            }
        }
        post_array.append(post)

    # Scrivi l'output in un nuovo file JSON
    with open(file_output, 'w', encoding='utf-8') as file:
        json.dump(post_array, file, indent=4, ensure_ascii=False)

# Esempio di utilizzo
file_ricette = '../recipes/RAW_recipes_images_ingredients_cleaned1.json'  # Sostituisci con il percorso del tuo file di ricette
file_output = 'posts_recipes.json'  # Sostituisci con il percorso del file di output desiderato

trasforma_ricette_in_post(file_ricette, file_output)

In [ ]:
with open('posts_recipes.json', 'r') as file:
    ricette = json.load(file)
    
for ricetta in ricette:
    for ingrediente in ricetta["recipe"]["ingredients"]:
        del ingrediente["Cals_per100grams"]
        
with open('posts_recipes_no_calories.json', 'w') as file:
    json.dump(ricette, file, indent=4)

Bisogna aggiungere comments e star_rankings e calcolare avg_star_ranking

Si aggiungono i comments

In [ ]:
# Leggi il file JSON dei post
with open('posts_recipes_no_calories.json', 'r') as file:
    posts = json.load(file)

# Leggi il file JSON dei commenti
with open('../comments/comments_cleaned.json', 'r') as file:
    comments = json.load(file)

# Crea una mappa dei commenti basata su user_id
comments_map = {}
for comment in comments:
    recipe_id = comment['recipe_id']
    if recipe_id not in comments_map:
        comments_map[recipe_id] = []
    comments_map[recipe_id].append(comment)

# Aggiungi i commenti ai post corrispondenti
for post in posts:
    recipe_id = post['id']
    post['comments'] = comments_map.get(recipe_id, [])

# Salva o restituisci il nuovo JSON
with open('posts_with_comments.json', 'w') as file:
    json.dump(posts, file, indent=4)


In [4]:
with open('posts_with_comments.json', 'r') as file:
    posts = json.load(file)
    
for post in posts:
    for comment in post["comments"]:
        del comment["recipe_id"]
        
with open('posts_with_comments_cleaned.json', 'w') as file:
    json.dump(posts, file, indent=4)

Si aggiungono gli star_rankings

In [5]:
# Leggi il file JSON dei post
with open('posts_with_comments_cleaned.json', 'r') as file:
    posts = json.load(file)

# Leggi il file JSON dei commenti
with open('../star_rankings/star_rankings_cleaned.json', 'r') as file:
    stars = json.load(file)

# Crea una mappa dei commenti basata su user_id
stars_map = {}
for star in stars:
    recipe_id = star['recipe_id']
    if recipe_id not in stars_map:
        stars_map[recipe_id] = []
    stars_map[recipe_id].append(star)

# Aggiungi i commenti ai post corrispondenti
for post in posts:
    recipe_id = post['id']
    post['starRankings'] = stars_map.get(recipe_id, [])

# Salva o restituisci il nuovo JSON
with open('posts_with_comments_stars.json', 'w') as file:
    json.dump(posts, file, indent=4)

In [6]:
with open('posts_with_comments_stars.json', 'r') as file:
    posts = json.load(file)
    
for post in posts:
    for comment in post["starRankings"]:
        del comment["recipe_id"]
        
with open('posts_with_comments_stars_cleaned.json', 'w') as file:
    json.dump(posts, file, indent=4)

Aggiungo avgStarRanking.

In [3]:
with open('posts_with_comments_stars_cleaned.json', 'r') as file:
    posts = json.load(file)

for post in posts:
    # Controlla se ci sono voti
    if 'starRankings' in post and post['starRankings']:
        total_votes = sum(vote['vote'] for vote in post['starRankings'])
        avg_ranking = str(round((total_votes / len(post['starRankings'])), 2))
    else:
        avg_ranking = "No votes available"

    # Aggiungi avgRanking al post
    post['avgRanking'] = avg_ranking
    
with open('posts_database.json', 'w') as file:
    json.dump(posts, file, indent=4)

Bisogna effettuare una modifica, avgRanking in realtà conviene che sia una stringa dato che va solo mostrata a video, in modo tale che se non sono presenti valutazioni tale campo puà avere anche come valore "No votes available"